Se utiliza la libreria de pandas para dar inicio a la estracción y tratamiento de los datos.

In [1]:
import pandas as pd
import numpy as np
import json
import ast

In [2]:
df = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')

C:\Users\anavi\AppData\Local\Temp\ipykernel_14380\737541241.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')


In [3]:
df.columns # Para mostrar las columnas que tiene el dataset original

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
df[df.duplicated(keep=False)].shape

(25, 24)

In [5]:
df = df.drop_duplicates()

In [6]:
df.shape

(45453, 24)

In [7]:
df.head(3) # Para mostrar el dataset original

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [8]:
# Se eliminan las columnas para aligerar el tamaño del dataset: video, imbd_id, poster_path y homepage (solicitadas)

df = df.drop(columns = ['adult', 'imdb_id', 'poster_path', 'homepage', 'video', ], errors='ignore')

df.columns # Para mostrar las columnas con las que se va a trabajar

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [9]:
# Se eliminan las columnas para aligerar el tamaño del dataset (no utilizadas)

df = df.drop(columns = ['homepage', 'original_language', 'overview','runtime', 'spoken_languages', 'status', 'tagline', 'production_countries'], errors='ignore')

df.columns # Para mostrar las columnas con las que se va a trabajar

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_title',
       'popularity', 'production_companies', 'release_date', 'revenue',
       'title', 'vote_average', 'vote_count'],
      dtype='object')

In [10]:
df.shape

(45453, 12)

Se empieza desde aqui a hacer la depuración de las columnas que se encuentran anidadas:
1. belongs_to_collection
2. genres
3. production_companies
4. production_countries

DEPURACIÓN COLUMNA 'belongs_to_collection'

In [11]:
# Se verifica si la columna tiene valores nulos

df['belongs_to_collection'].isnull().sum()

np.int64(40959)

In [12]:
# La columna tiene datos nulos, se reemplazan con listas vacías

df['belongs_to_collection'] = df['belongs_to_collection'].fillna('[]')
df['belongs_to_collection'].head(10)

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                   []
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                   []
4    {'id': 96871, 'name': 'Father of the Bride Col...
5                                                   []
6                                                   []
7                                                   []
8                                                   []
9    {'id': 645, 'name': 'James Bond Collection', '...
Name: belongs_to_collection, dtype: object

In [13]:
type(df['belongs_to_collection'][0])

str

In [14]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df['belongs_to_collection'] = df['belongs_to_collection'].apply(convertir_a_diccionario)

type(df['belongs_to_collection'][0])

dict

In [15]:
# Se muestran los tipos de datos únicos en la columna

df['belongs_to_collection'].apply(type).unique()

array([<class 'dict'>, <class 'list'>, <class 'float'>], dtype=object)

In [16]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df.iterrows():
    entry = row['belongs_to_collection']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_collection = pd.concat(expanded_dfs, ignore_index=True)

In [17]:
df_collection.columns

Index(['id', 'name', 'poster_path', 'backdrop_path'], dtype='object')

In [18]:
df_collection.shape

(4491, 4)

In [19]:
df_collection = df_collection.rename(columns={'name': 'Collection'}) #Cambio el nombre para dejarlo en el data frame final
df_collection = df_collection.drop_duplicates() #elimino los duplicados del df resultante
df_collection[df_collection['id'].duplicated(keep=False)] #Confirmo si quedan duplicados

,id,Collection,poster_path,backdrop_path


In [20]:
df = df.drop(columns=['belongs_to_collection']) #Elimino la columna que ya dupliqueé
df = pd.merge(df, df_collection[['id', 'Collection']], on='id', how='left') #Hago la combinación de datos

In [21]:
df.shape

(45453, 12)

In [22]:
df.columns

Index(['budget', 'genres', 'id', 'original_title', 'popularity',
       'production_companies', 'release_date', 'revenue', 'title',
       'vote_average', 'vote_count', 'Collection'],
      dtype='object')

In [23]:
df.head(1)

,budget,genres,id,original_title,popularity,production_companies,release_date,revenue,title,vote_average,vote_count,Collection
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection


DEPURACIÓN COLUMNA 'genres'

In [24]:
# Se verifica si la columna tiene valores nulos
 
df['genres'].isnull().sum()


np.int64(0)

In [25]:
type(df['genres'][0])

str

In [26]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df['genres'] = df['genres'].apply(convertir_a_diccionario)

type(df['genres'][0])

list

In [27]:
# Se muestran los tipos de datos únicos en la columna
df['genres'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [28]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df.iterrows():
    entry = row['genres']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_genres = pd.concat(expanded_dfs, ignore_index=True)

In [29]:
df_genres.shape

(91074, 3)

In [30]:
df_genres.head()

,id,name,order
0,862,Animation,1
1,862,Comedy,2
2,862,Family,3
3,8844,Adventure,1
4,8844,Fantasy,2


In [31]:
# Crear una función para combinar valores únicos en una lista
def combine_unique(series):
    return list(series.unique())

# Agrupar por 'id' y combinar valores únicos en listas para las columnas 'job' y otras relevantes
df_grouped = df_genres.groupby('id').agg({
    'name': combine_unique,
}).reset_index()

df_grouped.head()


,id,name
0,100,"[Comedy, Crime]"
1,10000,"[Comedy, Drama]"
2,10001,"[Comedy, Science Fiction]"
3,100010,"[Drama, War]"
4,100017,[Drama]


In [32]:
df.columns

Index(['budget', 'genres', 'id', 'original_title', 'popularity',
       'production_companies', 'release_date', 'revenue', 'title',
       'vote_average', 'vote_count', 'Collection'],
      dtype='object')

In [33]:
df = df.drop(columns=['genres'])
df_grouped = df_grouped.rename(columns={'name': 'genres'})
df = pd.merge(df, df_grouped[['id', 'genres']], on='id', how='left')

In [34]:
df.head(3)

,budget,id,original_title,popularity,production_companies,release_date,revenue,title,vote_average,vote_count,Collection,genres
0,30000000,862,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]"
1,65000000,8844,Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]"
2,0,15602,Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]"


In [35]:
df.shape

(45453, 12)

In [36]:
df.columns

Index(['budget', 'id', 'original_title', 'popularity', 'production_companies',
       'release_date', 'revenue', 'title', 'vote_average', 'vote_count',
       'Collection', 'genres'],
      dtype='object')

DEPURACIÓN COLUMNA 'production_companies'

In [37]:
# Se verifica si la columna tiene valores nulos
 
df['production_companies'].isnull().sum()

np.int64(3)

In [38]:
# La columna tiene datos nulos, se reemplazan con listas vacías

df['production_companies'] = df['production_companies'].fillna('[]')
df['production_companies'].head(10)

0       [{'name': 'Pixar Animation Studios', 'id': 3}]
1    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2    [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3    [{'name': 'Twentieth Century Fox Film Corporat...
4    [{'name': 'Sandollar Productions', 'id': 5842}...
5    [{'name': 'Regency Enterprises', 'id': 508}, {...
6    [{'name': 'Paramount Pictures', 'id': 4}, {'na...
7          [{'name': 'Walt Disney Pictures', 'id': 2}]
8    [{'name': 'Universal Pictures', 'id': 33}, {'n...
9    [{'name': 'United Artists', 'id': 60}, {'name'...
Name: production_companies, dtype: object

In [39]:
type(df['production_companies'][0])

str

In [40]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df['production_companies'] = df['production_companies'].apply(convertir_a_diccionario)

type(df['production_companies'][0])

list

In [41]:
# Se muestran los tipos de datos únicos en la columna
df['production_companies'].apply(type).unique()

array([<class 'list'>, <class 'bool'>], dtype=object)

In [42]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df.iterrows():
    entry = row['production_companies']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_prcompany = pd.concat(expanded_dfs, ignore_index=True)

In [43]:
df.shape

(45453, 12)

In [44]:
df_prcompany['order'].value_counts().sort_index()

order
1     33575
2     17001
3      9039
4      4325
5      2378
6      1428
7       867
8       553
9       373
10      240
11      173
12      131
13       97
14       78
15       62
16       53
17       32
18       25
19       22
20       17
21       12
22        8
23        5
24        5
25        4
26        3
Name: count, dtype: int64

In [45]:
df_prcompany.rename(columns={'name': 'Production_Company'}, inplace=True) #Cambio el nombre "name" por "Product_Company"
df_prcompany = df_prcompany[~df_prcompany['order'].between(4, 26)] #Elimino las filas para las peliculas que tengan más de 4 productoras.

In [46]:
df_prcompany = df_prcompany.drop_duplicates(subset=['id', 'order']) # Para eliminar duplicados
df_prcompany = df_prcompany.pivot(index='id', columns='order', values='Production_Company') # Crea una tabla pivote  para separar las filas en col
df_prcompany.columns = ['ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'] #Para dejar claros los nuevos nombres
df_prcompany.reset_index(inplace=True) #Se resetea el indice. 

In [47]:
df_prcompany.head()

,id,ProdCompany_1,ProdCompany_2,ProdCompany_3
0,100,Handmade Films Ltd.,Summit Entertainment,PolyGram Filmed Entertainment
1,10000,Ministère de la Culture et de la Francophonie,Ministère des Affaires Étrangères,NaN
2,10001,Warner Bros.,NaN,NaN
3,100010,Metro-Goldwyn-Mayer (MGM),NaN,NaN
4,100017,MMM Film Zimmermann & Co,NaN,NaN


In [48]:
df_prcompany.columns

Index(['id', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'], dtype='object')

In [49]:
df.columns

Index(['budget', 'id', 'original_title', 'popularity', 'production_companies',
       'release_date', 'revenue', 'title', 'vote_average', 'vote_count',
       'Collection', 'genres'],
      dtype='object')

In [50]:
df = df.drop(columns=['production_companies'])
df = pd.merge(df, df_prcompany[['id', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3']], on='id', how='left')

In [51]:
df.columns

Index(['budget', 'id', 'original_title', 'popularity', 'release_date',
       'revenue', 'title', 'vote_average', 'vote_count', 'Collection',
       'genres', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'],
      dtype='object')

In [52]:
df.shape

(45453, 14)

Una vez terminada la depuración de columnas anidadas, se verifican las columnas resultantes y la estructura del nuevo dataset. Se opta por no borrar columnas en esta etapa del proyecto, posteriormente se evaluará si se van a eliminar. 

In [53]:
df.columns # Para mostrar las columnas resultantes

Index(['budget', 'id', 'original_title', 'popularity', 'release_date',
       'revenue', 'title', 'vote_average', 'vote_count', 'Collection',
       'genres', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'],
      dtype='object')

In [54]:
df.head(3) # para mostrar la nueva estructura del dataset "movies_dataset.csv"

,budget,id,original_title,popularity,release_date,revenue,title,vote_average,vote_count,Collection,genres,ProdCompany_1,ProdCompany_2,ProdCompany_3
0,30000000,862,Toy Story,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",Pixar Animation Studios,NaN,NaN
1,65000000,8844,Jumanji,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]",TriStar Pictures,Teitler Film,Interscope Communications
2,0,15602,Grumpier Old Men,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]",Warner Bros.,Lancaster Gate,NaN


In [55]:
# Se reemplazan con cero los valores nulos en las columnas "revenue" y "budget" 
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

In [56]:
#Se eliminan los valores nulos de la columna "release_date"
df = df.dropna(subset=['release_date'])

In [57]:
#Se verifican como quedaron los cambios en estas dos columnas
df[['revenue', 'budget', 'release_date']].tail(20)

,revenue,budget,release_date
45432,0.0,0,1906-01-01
45433,0.0,0,1909-01-01
45434,0.0,0,1904-03-05
45435,0.0,0,1904-01-01
45436,0.0,0,2005-09-13
45437,0.0,0,1900-01-01
45438,0.0,0,1900-01-01
45439,0.0,0,1981-01-01
45440,0.0,0,2017-07-07
45441,0.0,0,2015-01-11


In [58]:
# Convertir la columna 'release_date' al formato de fecha AAAA-mm-dd
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
df['release_date'].head()

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]

In [59]:
# Crear la columna 'release_year' extrayendo el año de 'release_date' como cadena de texto
df['release_date'].fillna(pd.Timestamp('1900-01-01'), inplace=True)
df['release_year'] = df['release_date'].dt.year.astype(int).astype(str)
df['release_year'].head()

C:\Users\anavi\AppData\Local\Temp\ipykernel_14380\1895309714.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['release_date'].fillna(pd.Timestamp('1900-01-01'), inplace=True)


0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: object

In [60]:
# Asegurarse de que las columnas 'revenue' y 'budget' sean numéricas
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').fillna(0)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce').fillna(0)

# Crear la columna 'return' calculando el retorno de inversión
df['return'] = df.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] > 0 else 0, axis=1)

# Verificar las primeras filas de la nueva columna 'return'
print(df[['revenue', 'budget', 'return']].head())

       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000


In [61]:
df.head(3)

,budget,id,original_title,popularity,release_date,revenue,title,vote_average,vote_count,Collection,genres,ProdCompany_1,ProdCompany_2,ProdCompany_3,release_year,return
0,30000000.0,862,Toy Story,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",Pixar Animation Studios,NaN,NaN,1995,12.451801
1,65000000.0,8844,Jumanji,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]",TriStar Pictures,Teitler Film,Interscope Communications,1995,4.043035
2,0.0,15602,Grumpier Old Men,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]",Warner Bros.,Lancaster Gate,NaN,1995,0.000000


In [62]:
df.shape

(45366, 16)

In [63]:
df.columns

Index(['budget', 'id', 'original_title', 'popularity', 'release_date',
       'revenue', 'title', 'vote_average', 'vote_count', 'Collection',
       'genres', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3',
       'release_year', 'return'],
      dtype='object')

In [64]:
año_maximo = df['release_year'].max()
año_minimo = df['release_year'].min()

print(f"El año más reciente es: {año_maximo}")
print(f"El año más antiguo es: {año_minimo}")

El año más reciente es: 2020
El año más antiguo es: 1874


In [65]:
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce') #El año lo tengo como texto, lo paso a numero.
df_1990 = df[df['release_year'] <= 1990] # Filtro solo las peliculas hasta 1990 y creo un nuevo data frame


In [66]:
df_1990.shape

(16118, 16)

In [67]:
df_1990.to_csv('movies_1990.csv', index=False)

In [68]:
df = df[df['release_year'] > 1990] #Elimino todas las peliculas del df hasta 1980 del df original

In [69]:
df.shape

(29248, 16)

In [70]:
df.to_csv('movies_2020.csv', index=False)